In [3]:
import tensorflow as tf
import keras as kr

Using TensorFlow backend.


In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_df = pd.read_csv(r"C:\Users\anikp\Desktop\mnist_train.csv")
test_df = pd.read_csv(r"C:\Users\anikp\Desktop\mnist_test.csv")
n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [102]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

In [103]:
#Initializing the Hyperparameters
alpha = pow(10,-4)
dropout = 0.5

In [104]:
x = tf.placeholder('float',[None,n_input])
y = tf.placeholder('float',[None,n_output])

In [105]:
keep_drop = tf.placeholder(tf.float32)

In [106]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [107]:
layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2,weights['w3']),biases['b3'])
layer_drop = tf.nn.dropout(layer_3,keep_drop)
layer_out = tf.add(tf.matmul(layer_3,weights['out']),biases['out'])

In [108]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer_out))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [109]:
correct_pred = tf.equal(tf.argmax(layer_out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [110]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [116]:
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(328)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y, keep_drop:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={x: batch_x, y: batch_y, keep_drop:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

Iteration 0 	| Loss = 0.36443076 	| Accuracy = 0.91463417
Iteration 100 	| Loss = 0.21395381 	| Accuracy = 0.93292683
Iteration 200 	| Loss = 0.29342684 	| Accuracy = 0.93292683
Iteration 300 	| Loss = 0.21724412 	| Accuracy = 0.94512194
Iteration 400 	| Loss = 0.23762894 	| Accuracy = 0.9207317
Iteration 500 	| Loss = 0.18190983 	| Accuracy = 0.9512195
Iteration 600 	| Loss = 0.23522805 	| Accuracy = 0.9268293
Iteration 700 	| Loss = 0.29296666 	| Accuracy = 0.9207317
Iteration 800 	| Loss = 0.22607858 	| Accuracy = 0.9207317
Iteration 900 	| Loss = 0.20271899 	| Accuracy = 0.9268293


In [118]:
test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_drop:1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9247


In [115]:
import numpy as np
from PIL import Image

In [77]:
img = np.invert(Image.open(r"C:/Users/anikp/Desktop/test_img.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(layer_out,1), feed_dict={x: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2
